# Data Augmentation

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

In [ ]:
batch_size = 32
img_size = 299
train_path = '../data/sports/train/'
test_path = '../data/sports/test/'

## Data augmentation

If you get errors in this part, you may need to download the data first.

- Open a terminal
- `cd data/sports/`
- `python download_sport.py`

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=5,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
input_path = train_path + 'Beach volleyball/1e9ce0e76695de2e5d1f6964ab8c538.jpg'

img = image.load_img(input_path, target_size=(img_size, img_size))
img

In [ ]:
img_array = image.img_to_array(img)

In [ ]:
img_tensor = np.expand_dims(img_array, axis=0)

In [ ]:
plt.figure(figsize=(12, 12))

i = 0
for im in datagen.flow(img_tensor, batch_size=1):
    i += 1
    if i > 16:
        break
    plt.subplot(4, 4, i)
    plt.imshow(im[0])

## Exercise 1

`datagen` also has a method called `flow_from_directory`. Use it to produce a batch of 32 images of sports flowing from the training directory.

In [ ]:
train = datagen.flow_from_directory(train_path,
                                    target_size = (img_size, img_size),
                                    batch_size = batch_size,
                                    class_mode = 'binary')

In [ ]:
batch, labels = train.next()

In [ ]:
ld = {v: k for k, v in train.class_indices.items()}

In [ ]:
labels_s = [ld[i] for i in labels]

In [ ]:
plt.figure(figsize=(12, 12))
i = 1
for img, label in zip(batch, labels_s):
    plt.subplot(6, 6, i)
    plt.imshow(img)
    plt.title(label)
    i += 1

plt.tight_layout()

## Convolutional model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_size, img_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


## Exercise 2

Define a training and test data generators and use them to train the model using the `model.fit` function.

The training data generator should apply data augmentation while the test data generator should not modify the images except for rescaling them.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
history = model.fit(
              train_generator,
              steps_per_epoch=2100 // batch_size,
              epochs=4,
              validation_data=test_generator,
              validation_steps=900 // batch_size)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.legend(['train', 'test'])
plt.xlabel('Epochs')